In [9]:
import sys
sys.path.insert(0,"..")

from src.utils import read_file_with_custom_data_types

df = read_file_with_custom_data_types("../data/transformed/transform.csv")

df.head()

,id,created_at,updated_at,name,description,gsf_standards_version,estimated_annual_credits,crediting_period_start_date,crediting_period_end_date,methodology,...,Goal_2,Goal_11,Goal_9,Goal_10,Goal_16,Goal_14,crediting_days,VER_sold_percentage,VER_sold_percentage_per_day,continent_code
0,3653,2022-07-25 14:23:52+00:00,2023-03-15 01:54:01+00:00,GS10789 VPA10: Efficient and Clean Cooking for...,This VPA deploys highly efficient improved cha...,Gold Standard for the Global Goals,311327,2021-03-30,2026-03-30,GS TPDDTEC v3.1,...,False,False,False,False,False,False,1826,0.0,0.0,AF
1,3649,2022-07-20 14:54:25+00:00,2023-03-27 18:37:46+00:00,TASC Clean Cooking PoA – VPA 3 (Zambia),The goal of the proposed project is to distrib...,Gold Standard for the Global Goals,96484,2021-09-21,2026-09-20,GS TPDDTEC v3.1,...,False,False,False,False,False,False,1825,0.0,0.0,AF
2,3595,2022-06-23 05:52:18+00:00,2023-03-28 12:23:43+00:00,GS10818 - Dissemination of Improved Cookstoves...,The project involves providing access to clean...,Gold Standard for the Global Goals,39126,2022-03-01,2027-03-01,AMS-II.G. Energy Efficiency Measures in Therma...,...,False,False,False,False,False,False,1826,0.0,0.0,AS
3,3594,2022-06-23 05:52:03+00:00,2023-03-28 12:23:07+00:00,GS10818 - Dissemination of Improved Cookstoves...,The project involves providing access to clean...,Gold Standard for the Global Goals,39126,2022-01-21,2027-01-21,AMS-II.G. Energy Efficiency Measures in Therma...,...,False,False,False,False,False,False,1826,0.0,0.0,AS
4,3593,2022-06-23 05:51:46+00:00,2023-03-28 12:22:37+00:00,GS10818 - Dissemination of Improved Cookstoves...,The project involves providing access to clean...,Gold Standard for the Global Goals,39126,2021-12-20,2026-12-20,AMS-II.G. Energy Efficiency Measures in Therma...,...,False,False,False,False,False,False,1826,0.0,0.0,AS
